<a href="https://colab.research.google.com/github/BogdanTurbal/simple_protein_biologial_function_prediction_model/blob/main/proteins_submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
# Required for progressbar widget
import progressbar

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

#Load data


In [ ]:
train_terms = pd.read_csv("/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv",sep="\t")
# Set the limit for label
num_of_labels = 1500
# Take value counts in descending order and fetch first 1500 `GO term ID` as labels
labels = train_terms['term'].value_counts().index[:num_of_labels].tolist()

#Define model

In [ ]:
INPUT_SHAPE = [1024]#[train_df.shape[1]]
BATCH_SIZE = 512

model = tf.keras.Sequential([
    #tf.keras.layers.BatchNormalization(input_shape=INPUT_SHAPE),
    tf.keras.layers.Dense(units=896, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=896, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=768, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=768, activation='relu'),
    tf.keras.layers.Dense(units=num_of_labels, activation='sigmoid')
])


# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.Accuracy(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()]
)
_ = model(tf.zeros((1, 1024)))
model.load_weights('/kaggle/input/model-weights-3/model_v_3_0 (1).h5')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (1, 896)                  918400    
                                                                 
 dropout (Dropout)           (1, 896)                  0         
                                                                 
 batch_normalization (BatchN  (1, 896)                 3584      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (1, 896)                  803712    
                                                                 
 dropout_1 (Dropout)         (1, 896)                  0         
                                                                 
 batch_normalization_1 (Batc  (1, 896)                 3584      
 hNormalization)                                        

In [ ]:
test_embeddings = np.load('/kaggle/input/t5embeds/test_embeds.npy')

# Convert test_embeddings to dataframe
column_num = test_embeddings.shape[1]
test_df = pd.DataFrame(test_embeddings, columns = ["Column_" + str(i) for i in range(1, column_num+1)])
print(test_df.shape)

(141865, 1024)


In [ ]:
predictions =  model.predict(test_df)

4434/4434 [==============================] - 34s 8ms/step


In [ ]:
df_submission = pd.DataFrame(columns = ['Protein Id', 'GO Term Id','Prediction'])
test_protein_ids = np.load('/kaggle/input/t5embeds/test_ids.npy')
l = []
for k in list(test_protein_ids):
    l += [ k] * predictions.shape[1]

df_submission['Protein Id'] = l
df_submission['GO Term Id'] = labels * predictions.shape[0]
df_submission['Prediction'] = predictions.ravel()
df_submission.to_csv("submission.tsv",header=False, index=False, sep="\t")

In [ ]:
df_submission

,Protein Id,GO Term Id,Prediction
0,Q9CQV8,GO:0005575,6.567487e-01
1,Q9CQV8,GO:0008150,6.297429e-01
2,Q9CQV8,GO:0110165,6.341953e-01
3,Q9CQV8,GO:0003674,7.591765e-01
4,Q9CQV8,GO:0005622,4.251500e-01
...,...,...,...
212797495,A0A3G2FQK2,GO:0051783,5.779548e-03
212797496,A0A3G2FQK2,GO:0031674,3.634445e-04
212797497,A0A3G2FQK2,GO:0001818,4.518046e-05
212797498,A0A3G2FQK2,GO:0006874,1.995342e-06
